In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pm4py

In [0]:
!ls '/content/drive/My Drive/Eventlog/Benchmark/seperate file/WRI'


BPIC12.xes     BPIC13_i.xes	       RTFMP.xes
BPIC13_cp.xes  BPI_Challenge_2012.xes  SEPSIS.xes


In [0]:
#Importing library to read log file
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.objects.log.exporter.csv import factory as csv_exporter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.objects.log.exporter.xes import factory as xes_exporter
from pm4py.algo.filtering.pandas.variants import variants_filter

from datetime import datetime
import pandas as pd
import operator
import re

In [0]:
#Reading XES fileafter 

filename = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/WRI/SEPSIS.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = "/content/drive/My Drive/Eventlog/Benchmark/seperate file/no_need9.csv"

csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df_sepsis = pd.read_csv(fileName)

# Filtering only three columns
df_clean = df_sepsis.loc[:,['case:concept:name','concept:name','time:timestamp']]

In [52]:
df_clean.head(20)
#len(df_clean)

case:concept:name      concept:name             time:timestamp
0                  A   ER Registration  2014-10-22 11:15:41+02:00
1                  A        Leucocytes  2014-10-22 11:27:00+02:00
2                  A               CRP  2014-10-22 11:27:00+02:00
3                  A        LacticAcid  2014-10-22 11:27:00+02:00
4                  A         ER Triage  2014-10-22 11:33:37+02:00
5                  A  ER Sepsis Triage  2014-10-22 11:34:00+02:00
6                  A         IV Liquid  2014-10-22 14:03:47+02:00
7                  A    IV Antibiotics  2014-10-22 14:03:47+02:00
8                  A      Admission NC  2014-10-22 14:13:19+02:00
9                  A               CRP  2014-10-24 09:00:00+02:00
10                 A        Leucocytes  2014-10-24 09:00:00+02:00
11                 A        Leucocytes  2014-10-26 08:00:00+01:00
12                 A               CRP  2014-10-26 08:00:00+01:00
13                 A        Leucocytes  2014-10-28 08:00:00+01:00
14                 A               CRP  2014-10-28 08:00:00+01:00
15                 A               CRP  2014-10-30 08:00:00+01:00
16                 A        Leucocytes  2014-10-30 08:00:00+01:00
17                 A        Leucocytes  2014-10-31 08:00:00+01:00
18                 A               CRP  2014-10-31 08:00:00+01:00
19                 A               CRP  2014-11-02 08:00:00+01:00

In [0]:
df_clean["case:concept:name"].fillna("KMM",inplace=True)

In [7]:
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
  c +=1
print(c)

16


In [0]:
act_rename = ["A01","A02","A03","A04","A05","A06","A07","A08","A09","A10","A11","A12","A13","A14","A15","A16"]                     
act_dic = {}
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
  act_dic[i] = act_rename[c]
  c +=1
  
def activity_change(row):
  rename = act_dic[row]
  return rename
df_clean["concept:name"] = df_clean["concept:name"].apply(activity_change)

# The following function change time format
def time_change(row):
  
  if len(row) == 32:
    dateTime = row[:19]
    zone = row[26:]
    newTime = dateTime+zone
    return newTime
  else:
    return row

df_clean["time:timestamp"] = df_clean["time:timestamp"].apply(time_change)
df_clean["time:timestamp"] = pd.to_datetime(df_clean["time:timestamp"],)
df_clean["case:concept:name"] = df_clean["case:concept:name"].astype(str)


In [56]:
df_clean.loc[df_clean["case:concept:name"] == "A" , :]

case:concept:name concept:name             time:timestamp
0                  A          A01  2014-10-22 11:15:41+02:00
1                  A          A02  2014-10-22 11:27:00+02:00
2                  A          A03  2014-10-22 11:27:00+02:00
3                  A          A04  2014-10-22 11:27:00+02:00
4                  A          A05  2014-10-22 11:33:37+02:00
5                  A          A06  2014-10-22 11:34:00+02:00
6                  A          A07  2014-10-22 14:03:47+02:00
7                  A          A08  2014-10-22 14:03:47+02:00
8                  A          A09  2014-10-22 14:13:19+02:00
9                  A          A03  2014-10-24 09:00:00+02:00
10                 A          A02  2014-10-24 09:00:00+02:00
11                 A          A02  2014-10-26 08:00:00+01:00
12                 A          A03  2014-10-26 08:00:00+01:00
13                 A          A02  2014-10-28 08:00:00+01:00
14                 A          A03  2014-10-28 08:00:00+01:00
15                 A          A03  2014-10-30 08:00:00+01:00
16                 A          A02  2014-10-30 08:00:00+01:00
17                 A          A02  2014-10-31 08:00:00+01:00
18                 A          A03  2014-10-31 08:00:00+01:00
19                 A          A03  2014-11-02 08:00:00+01:00
20                 A          A02  2014-11-02 08:00:00+01:00
21                 A          A10  2014-11-02 15:15:00+01:00

In [0]:
# Save all trace id and activity belongs to a trace into dictionary
def activity(df_eve,unique_id):
  
  traces = {}
  
  for i in unique_id:
    list_a = []
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    for n in range(len(row_filter)):
      list_a.append(row_filter.loc[row_filter['case:concept:name']==i,'concept:name'][n])
    traces[i] = list_a
  return traces


# Filtering Distinct trace only
def dist_trace(df_eve,unique_id):
  
  Distinct_trace = []
  Distinct_trace_id = []
  # Calling activity method
  traces_list = activity(df_eve,unique_id)
  
  for k, v in traces_list.items():
      if v not in Distinct_trace:
        Distinct_trace.append(v)
        Distinct_trace_id.append(k)
        
  return Distinct_trace_id,Distinct_trace

unique_id = df_clean["case:concept:name"].unique()
dist_trace_id, dist_traces = dist_trace(df_clean,unique_id)

In [0]:

# Activity comination
def combination_of_activity(act_rename):
  # The following functions gets combination of events
  activity_name = act_rename

  Activity_Name = []
  for i in range(len(activity_name)):
    for c in range(len(activity_name)):
      Activity_Name.append([activity_name[i],activity_name[c]])
      #Activity_Name.append([activity_name[c],activity_name[i]])

  Activity_combination = []
  for combination in Activity_Name:
    if combination not in Activity_combination:
      Activity_combination.append(combination)
      
  return Activity_combination


# Count the combination
def combination_counter(trace_list,activity_combination):
  counts_of_combination = {}

  for trace_activity in trace_list:
  
    Activity_name = []
    for i in range(len(trace_activity)-1):
        Activity_name.append([trace_activity[i],trace_activity[i+1]])

    for c in Activity_name:
      if c in activity_combination:
        k =  repr(c)
        if k not in counts_of_combination.keys():
          counts_of_combination[k] = 1
        else:
          counts_of_combination[k] += 1
        
  return counts_of_combination


# Frequency of single activity
def activity_count(df_eve,unique_id,fre_act):
  
  Freq_act = 0
  for i in unique_id:
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    
    if fre_act:
      len_act = len(row_filter.loc[row_filter["concept:name"]==fre_act,"concept:name"])
      Freq_act +=len_act
    
  return Freq_act

freq_activity = {}
for i in act_rename:
  frequency_act = activity_count(df_clean,unique_id,i)
  freq_activity[i] = frequency_act
  #print("Frequency of {}:{}".format(i,frequency_act))
  
Activity_combi = combination_of_activity(act_rename)
CountsofComb = combination_counter(dist_traces,Activity_combi)

In [0]:
CountsofComb

In [0]:
def prob_of_activity(CountsofComb,freq_activity,threshold_value):
  ab_a = {}
  ab_b = {}
  similar = {}

  #sorted_d = dict( sorted(CountsofComb.items(), key=operator.itemgetter(1),reverse=True))

  # finding the probability of activity given a
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[2:5]]),3)  
      if prob >= threshold_value:
        a = repr([k[2:5],k[9:12]])
        ab_a[a] = prob
        #print(k, ":", prob)
  #print()

  # finding the probability of acitivity given b
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[9:12]]),3)
      if prob >= threshold_value:
        b = repr([k[2:5],k[9:12]])
        ab_b[b] = prob
        #print(k, ":", prob)      

  for i in ab_a.keys():
    if i in ab_b.keys():
      c = repr([i[2:5],i[9:12]])
      prob_add = ab_a[i] + ab_b[i]
      similar[c] = prob_add 

  similar = dict( sorted(similar.items(), key=operator.itemgetter(1),reverse=True))
  similar_keys = list(similar.keys())
  similar_values = similar.values()
  return similar_keys,similar_values

In [0]:
def event_dropper(df_eve, activity_name, new_activity_name):
   
  unique_id = df_eve["case:concept:name"].unique()
  df_final = pd.DataFrame(columns=df_eve.columns)
  
  for i in unique_id: 
    event_index = []
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
      
    
    for event in range(len(row_filter.loc[:,'concept:name'])-1):
            
      if activity_name[2:5] == activity_name[9:12]:
        if (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
          event_index.append(event)
        
      elif (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
        event_index.append(event)
        row_filter.loc[event + 1,'concept:name']=row_filter.loc[event + 1,'concept:name'].replace(activity_name[9:12],new_activity_name)
       

    row_filter.drop(row_filter.index[event_index],inplace=True)
    df_final = pd.concat([df_final, row_filter],ignore_index=True)

  return df_final

In [0]:
def remove_self_loop(df,file_Num,replace_activity):
     
  same_act = []
  act_rename = df["concept:name"].unique()
  unique_id = df["case:concept:name"].unique()
  dist_trace_id, dist_traces = dist_trace(df,unique_id)
  Activity_combi = combination_of_activity(act_rename)
  CountsofComb1 = combination_counter(dist_traces,Activity_combi)

  sorted_df = dict( sorted(CountsofComb1.items(), key=operator.itemgetter(1),reverse=True))
  for k , v in sorted_df.items():
    #print(k, ":", v)
    if k[2:5]==k[9:12]:
      same_act.append(k)
      #print(k, ":", v)
  
  new_a = same_act[0]
  df1 = event_dropper(df, new_a, None)
  for a in range(1,len(same_act)):
    new_ac = same_act[a]
    df1 = event_dropper(df1,new_ac,None)
    
  # Converting csv file to log file
  log = conversion_factory.apply(df1)
  # Saving log into xes file
  xes_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/SEPSIS_{}.xes'.format(file_Num)
  xes_exporter.export_log(log, xes_file)


In [0]:
import numpy as np
threshold_values = np.arange(0.1, 1, 0.1)
n = 1
replace_activity = {}

for t_v in threshold_values:
  similar_activity,similar_prob = prob_of_activity(CountsofComb,freq_activity,0.5)
  #print(list(similar_activity))
  #break
  if len(similar_activity)>=1:
    
    # first combination
    if similar_activity[0][2:5] == similar_activity[0][9:12]:      
      df = event_dropper(df_clean, similar_activity[0],None)
    else:
      df = event_dropper(df_clean, similar_activity[0],"BBB")
      for ak in [similar_activity[0][2:5], similar_activity[0][9:12]]: 
        if ak not in replace_activity.keys():
          replace_activity[ak] = "BBB"
      
    # Remaining combination
    for i in range(1,len(similar_activity)):
      if similar_activity[i][2:5] == similar_activity[i][9:12]:
        df = event_dropper(df, similar_activity[i],None)
      else:
        df = event_dropper(df, similar_activity[i],"BB{}".format(i))
        for ak in [similar_activity[i][2:5], similar_activity[i][9:12]]:
          if ak not in replace_activity.keys():
            replace_activity[ak] = "BB{}".format(i)
            
  for rk in replace_activity.keys():
    df["concept:name"] = df["concept:name"].apply(lambda x: re.sub( rk, replace_activity[rk] , x))

  remove_self_loop(df,n,replace_activity)
  break
  n +=1
  

In [78]:
similar_activity

["['A01', 'A05']", "['A05', 'A06']", "['A07', 'A08']"]

In [0]:
df1.loc[df1["case:concept:name"] == "A" , :] 

In [74]:
df["concept:name"].unique()

array(['BBB', 'BB4', 'BB8', 'BB1', 'BB3', 'BB5', 'BB2', 'A12', 'A13',
       'A14', 'A15', 'A16'], dtype=object)

In [29]:
len(df["concept:name"].unique()

array(['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09',
       'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16'], dtype=object)

In [0]:
#Reading XES fileafter 
filename = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/SEPSIS_1.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = "/content/drive/My Drive/Eventlog/Benchmark/seperate file/no_need10.csv"
csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df_sepsis1 = pd.read_csv(fileName)

In [83]:
df_sepsis1.head()

case:concept:name concept:name             time:timestamp
0                 A          BBB  2014-10-22 11:15:41+02:00
1                 A          A02  2014-10-22 11:27:00+02:00
2                 A          A03  2014-10-22 11:27:00+02:00
3                 A          A04  2014-10-22 11:27:00+02:00
4                 A          BB1  2014-10-22 11:34:00+02:00

In [84]:
len(df_sepsis1["concept:name"].unique())

14

In [0]:
unique_id = df_sepsis1["case:concept:name"].unique()
dist_trace_id, dist_traces = dist_trace(df_sepsis1,unique_id)

In [86]:
len(dist_traces)

756